# Part 2: Create agent (Driver notebook)
This is very similar to an auto-generated notebook created by an AI Playground export. There are three notebooks in the same folder as a set:
1. [agent]($./agent): contains the code to build the agent (only the code in `mlflow.models.set_model` will be served)
2. [**driver**]($./driver): references the agent code then logs, registers, evaluates and deploys the agent.
3. [config.yml]($./config.yml): contains the configuration settings.

This notebook uses [Mosaic AI Agent Framework](https://docs.databricks.com/en/generative-ai/retrieval-augmented-generation.html) to create your agent. It defines a supervisory LangChain agent that decides which other 4 ReAct agents to assign tasks too. These 4 agents are:
1. **SQL agent** that can run SQL functions (including batch AI functions) as tools
2. **Calculator agent** that can run python code for mathematical calculations
3. **Genie agent** that can do Q&A on structured table(s) using natural language
4. **Retriever agent** that can do Q&A on unstructured text in a Vector Store
![](../graph.png)

 **_NOTE:_**  This notebook uses LangChain, however Mosaic AI Agent Framework is [compatible](%md

## Prerequisites
1. Create the required tools (SQL functions, UC functions, Genie Space, Vector Store) in Part 1.
2. Check  [config.yml]($./config.yml) settings

## Next steps
After evaluating and serving your agent in this notebook, try out your agent in AI Playground or Model Serving.

In [0]:
%pip install -U -qqqq databricks-agents langgraph==0.4.3 langgraph-supervisor==0.0.21 langgraph-checkpoint==2.0.25 langchain==0.3.25 langchain_core==0.3.59 langchain-community==0.3.24 pydantic==2.11.4 databricks-sdk==0.52.0 mlflow-skinny==2.22.0 databricks-langchain==0.5.0 databricks-vectorsearch==0.56
# databricks-agents mlflow langchain==0.2.16 langgraph-checkpoint==1.0.12  langchain_core langchain-community==0.2.16 langgraph==0.2.16 pydantic langchain_databricks
dbutils.library.restartPython()

In [0]:
%pip freeze > requirements_driver.txt

In [0]:
import mlflow

def get_latest_model_version(model_name):
    from mlflow.tracking import MlflowClient
    mlflow_client = MlflowClient(registry_uri="databricks-uc")
    latest_version = 1
    for mv in mlflow_client.search_model_versions(f"name='{model_name}'"):
        version_int = int(mv.version)
        if version_int > latest_version:
            latest_version = version_int
    return latest_version

In [0]:
from mlflow.models import ModelConfig

mlflow.langchain.autolog()
mlflow.set_registry_uri('databricks-uc')

config = ModelConfig(development_config="config.yml")
config.to_dict()

In [0]:
catalog_name = config.get("catalog")
schema_name = config.get("schema")
model_name = "customer_service"

registered_name = f"{catalog_name}.{schema_name}.{model_name}"
artifact_path = "agent"

In [0]:
query = "Can you give me some troubleshooting steps for SoundWave X5 Pro Headphones that won't connect?"
input_example = {
    "messages": [
        {
            "role": "user",
            "content": query
        }
    ]
}

### Log the `agent` as an MLflow model
Log the agent as code from the [agent]($./agent) notebook. See [MLflow - Models from Code](https://mlflow.org/docs/latest/models.html#models-from-code).

In [0]:
# Log the model to MLflow
import os
from mlflow.models.resources import (
    DatabricksVectorSearchIndex, 
    DatabricksServingEndpoint,
    DatabricksFunction,
    DatabricksGenieSpace,
    DatabricksTable
)

with mlflow.start_run():
    logged_agent_info = mlflow.langchain.log_model(
        # in agent_noextras, VS is code, not a fn
        lc_model=os.path.join(os.getcwd(), 'agent'),
        artifact_path=artifact_path,
        registered_model_name=registered_name,
        model_config="config.yml",
        pip_requirements=[
            "langchain==0.3.25",
            "langchain-community==0.3.24",
            "langgraph==0.4.3",
            "langgraph-supervisor==0.0.21",
            "langgraph-prebuilt==0.1.8",
            "langgraph-checkpoint==2.0.25",
            "pydantic==2.11.4",
            "databricks_langchain==0.5.0",
            "databricks-vectorsearch==0.56",
            "ipython"
        ],
        input_example=input_example,
        # specify resources for deployed server to have explicit access
        resources=[
            DatabricksServingEndpoint(endpoint_name=config.get("llm_endpoint")),
            DatabricksVectorSearchIndex(index_name=config.get('retriever')['vs_index']),
            DatabricksFunction(function_name="yen_training.agents.get_latest_interaction"),
            DatabricksFunction(function_name="yen_training.agents.get_requests_history"),
            DatabricksFunction(function_name="yen_training.agents.get_return_policy"),
            DatabricksFunction(function_name="yen_training.agents.extract_product"),
            DatabricksFunction(function_name="system.ai.python_exec"),
            DatabricksGenieSpace(genie_space_id=config.get("genie_space_id")),
            DatabricksTable(table_name=config.get("genie_table")),
        ]
    )

In [0]:
model_uri = f"runs:/{logged_agent_info.run_id}/{artifact_path}"
#model_uri = 'runs:/a7927c82ca514389ba8da34825984c53/agent'

## Evaluate the agent with [Agent Evaluation](https://docs.databricks.com/generative-ai/agent-evaluation/index.html)

### Curate a Q&A evaluation dataset

In [0]:
import pandas as pd

data = {
    "request": [
        "What color options are available for the Aria Modern Bookshelf?",
        "How should I clean the Aurora Oak Coffee Table to avoid damaging it?",
        "How should I clean the BlendMaster Elite 4000 after each use?",
        "How many colors is the Flexi-Comfort Office Desk available in?",
        "What sizes are available for the StormShield Pro Men's Weatherproof Jacket?",
        "What should I do if my SmartX Pro device won’t turn on?",
        "How many people can the Elegance Extendable Dining Table seat comfortably?",
        "What colors is the Urban Explorer Jacket available in?",
        "What is the water resistance rating of the BrownBox SwiftWatch X500?",
        "What colors are available for the StridePro Runner?"
    ],
    "expected_facts": [
        [
            "The Aria Modern Bookshelf is available in natural oak finish",
            "The Aria Modern Bookshelf is available in black finish",
            "The Aria Modern Bookshelf is available in white finish"
        ],
        [
            "Use a soft, slightly damp cloth for cleaning.",
            "Avoid using abrasive cleaners."
        ],
        [
            "The jar of the BlendMaster Elite 4000 should be rinsed.",
            "Rinse with warm water.",
            "The cleaning should take place after each use."
        ],
        [
            "The Flexi-Comfort Office Desk is available in three colors."
        ],
        [
            "The available sizes for the StormShield Pro Men's Weatherproof Jacket are Small, Medium, Large, XL, and XXL."
        ],
        [
            "Press and hold the power button for 20 seconds to reset the device.",
            "Ensure the device is charged for at least 30 minutes before attempting to turn it on again."
        ],
        [
            "The Elegance Extendable Dining Table can comfortably seat 6 people."
        ],
        [
            "The Urban Explorer Jacket is available in charcoal, navy, and olive green"
        ],
        [
            "The water resistance rating of the BrownBox SwiftWatch X500 is 5 ATM."
        ],
        [
            "The colors available for the StridePro Runner should include Midnight Blue.",
            "The colors available for the StridePro Runner should include Electric Red.",
            "The colors available for the StridePro Runner should include Forest Green."
        ]
    ]
}

eval_dataset = pd.DataFrame(data)
display(eval_dataset)

### [OPTIONAL] Generate synthetic Q&A evaluation dataset

In [0]:
# Use the synthetic eval generation API to get some evals
from databricks.agents.evals import generate_evals_df

# Documents to generate synthetic Q&A grounded on the document context
# should be in a Pandas or Spark DataFrame with columns `content STRING` and `doc_uri STRING`.
docs = spark.table("yen_training.agents.product_docs") \
  .withColumnsRenamed({"product_id": "doc_uri",
                       "product_doc": "content"})
  
# "Ghost text" for agent description and question guidelines - feel free to modify as you see fit.
agent_description = f"""
The agent is a RAG chatbot that answers product questions using the product documentation"""
question_guidelines = f"""
# User personas
- A customer asking about product specifications and how to use a product
- A customer support assistant asking how to address customer's questions

# Example questions
- How to troubleshooting connecting to wireless headphones via bluetooth?
- What is the weight of a Macbook Pro?

# Additional Guidelines
- Questions should human-like and not repeat the same information.
"""

eval_dataset = generate_evals_df(
    docs=docs,  # Pass your docs. 
    num_evals=1000,  # How many synthetic evaluations to generate
    agent_description=agent_description,
    question_guidelines=question_guidelines,
)
display(eval_dataset)

In [0]:
spark.createDataFrame(eval_dataset) \
    .write.mode("overwrite") \
    .saveAsTable("yen_training.agents.syn_eval_dataset")

In [0]:
import mlflow
import pandas as pd

with mlflow.start_run(run_id=logged_agent_info.run_id):
    eval_results = mlflow.evaluate(
        model_uri,  # replace `chain` with artifact_path that you used when calling log_model.
        data=eval_dataset,  # Your evaluation dataset
        model_type="databricks-agent",  # Enable Mosaic AI Agent 
    )

# Review the evaluation results in the MLFLow UI (see console output), or access them in place:
display(eval_results.tables['eval_results'])

## Change our retriever `k` to 2 in [config.yml]($./config.yml) 
Avoid retrieving too many contradictory documents

In [0]:
from IPython.display import display, HTML

# Retrieve the Databricks host URL
workspace_url = spark.conf.get('spark.databricks.workspaceUrl')

# Create HTML link to created agent
html_link = f'<a href="https://{workspace_url}/explore/data/models/{catalog_name}/{schema_name}/{model_name}" target="_blank">Go to Unity Catalog to see Registered Agent</a>'
display(HTML(html_link))

## Deploy the agent

In [0]:
from databricks import agents

latest_version = get_latest_model_version(registered_name)
latest_version

In [0]:
# Deploy the model to the review app and a model serving endpoint
agents.deploy(model_name=registered_name, 
              model_version=latest_version,
              scale_to_zero=True,
              tags = {"endpointSource": "playground"})

## [OPTIONAL] For testing inferencing locally
Useful to test inferencing before remote deployment

In [0]:
# Test a question best answered by the retriever
loaded_model = mlflow.langchain.load_model(model_uri)
loaded_model.invoke(input_example)

In [0]:
# Test a question best answered by Genie (chat with Customer Service table)
example = {
    "messages": [
        {
            "role": "user",
            "content": "In which month do we have the most customer requests?"
        }
    ]
}
loaded_model.invoke(example)

In [0]:
# Test a question best answered by the function tools esp calculator (without CoT)
example = {
    "messages": [
        {
            "role": "user",
            "content": "For the customer named Tina Daugherty, how much in costs has been incurred assuming that each customer return costs $20, each technical support interaction cost $10 and each product inqury costs $5?"
        }
    ]
}
loaded_model.invoke(example)

In [0]:
# Test a question best answered by the function tools esp calculator (with CoT)
example = {
    "messages": [
        {
            "role": "user",
            "content": "For the customer named Tina Daugherty, how much in costs has been incurred assuming that each customer return costs $20, each technical support interaction cost $10 and each product inqury costs $5? To do this, first get the request history for Tina Daugherty returning the number of the returns, technical support interactions and product inquries and then compute the total cost with a calculator."
        }
    ]
}
loaded_model.invoke(example)

In [0]:
# Test a question best answered by the function tools esp ai_extract
example = {
    "messages": [
        {
            "role": "user",
            "content": "Which is the most frequently mentioned product in the issue descriptions? First query issue_description in customer service data, then apply the extract_product tool on the issue_description. Answer with only the product name and nothing else."
        }
    ]
}
loaded_model.invoke(example)